# TD/TP : Ricochet Robots - 碰撞機器人

<br>
<br>

Le jeu Ricochet Robots consiste à déplacer des robots sur un plateau composé d'une grille de cases séparées par quelques murs. Chaque robot se déplace en ligne droite et avance toujours jusqu'au premier mur qu'il rencontre. À chaque tour, le but est d'amener un robot sur une case objectif fixée en utilisant le moins de mouvements possibles.

Un exemple en 8 mouvements est représenté dans l'image ci-dessous.


<img src="exemple.png" width="500">

Précision : s'il n'y a pas de mur juste après l'objectif, le robot doit poursuivre son mouvement sans pouvoir s'arrêter sur la case objectif. En particulier, le jeu n'a pas de solution si l'objectif est sur une case qui n'est entourée par aucun mur.


Ce TD/TP propose de modéliser le jeu Ricochet Robots à l'aide d'un graphe afin de résoudre le jeu et de répondre à plusieurs questions en rapport.


## Construction des sommets

<br>

Les lignes et les colonnes du plateau sont numérotées de 0 à 15 en partant du coin en haut à gauche. Chaque case peut ainsi être modélisée par la liste de ses coordonnées. Par exemple, dans l'image précédente, le robot démarre de la case `[5,12]` pour atteindre l'objectif situé sur la case `[10,5]`. Son déplacement en 8 mouvements peut se décomposer de la façon suivante :

`[5,12]` $\rightarrow$ `[0,12]` $\rightarrow$ `[0,10]` $\rightarrow$ `[13,10]` $\rightarrow$ `[13,2]` $\rightarrow$ `[15,2]` $\rightarrow$ `[15,4]` $\rightarrow$ `[10,4]` $\rightarrow$ `[10,5]`.

1. Trouver une solution si le robot démarre de la case `[4,5]` et souhaite atteindre un objectif situé sur la case `[13,14]`. Combien de mouvements avez-vous utilisés ?

On souhaite modéliser le plateau à l'aide d'un graphe dont chaque sommet représente une case accessible par les robots (c'est-à-dire toutes les cases du plateau sauf les 4 cases centrales).

2. Construire une liste `Sommets` dans laquelle la liste des coordonnées de chaque case accessible du plateau apparaît une seule fois. Vérifier que l'ordre du graphe obtenu avec `len(Sommets)` est égal à `252`.

In [1]:
Sommets=[]
for i in range(16):
    for j in range(16):
        Sommets=Sommets+[[i,j]]
Sommets.remove([7,7])
Sommets.remove([7,8])
Sommets.remove([8,7])
Sommets.remove([8,8])

In [2]:
nb_cases=len(Sommets)
print(nb_cases)

252


3. Écrire une fonction `indice_case` qui prend en argument la liste `[i,j]` des coordonnées d'une case, puis qui renvoie son indice dans `Sommets`. Si `[i,j]` ne correspond pas à une case accessible, `indice_case([i,j])` doit renvoyer `None`.

In [3]:
def indice_case(case):
    for indice in range(nb_cases):
        if Sommets[indice]==case:
            return indice
    return None

## Construction de la liste d'adjacence après un pas

<br>

Le but de cette partie est de construire la liste d'adjacence après un seul pas du déplacement du robot (le mouvement entier jusqu'au premier mur sera modélisé dans la partie suivante). Par exemple, les sommets `[1,7]` et `[1,8]` sont considérés adjacents, alors que les sommets `[2,5]` et `[2,6]` ne le sont pas, ni les sommets `[3,0]` et `[3,4]`.

Pour cela, on va modéliser les murs en numérotant de 0 à 16 les droites horizontales sépararant les lignes et les droites verticales séparant les colonnes, en partant du coin en haut à gauche comme dans l'image ci-dessous.

<img src="numerotation.png" width="400">

À l'aide de cette numérotation, on peut faire la liste des murs horizontaux et des murs verticaux. Par exemple dans la première colonnne, on croise 4 murs horizontaux : sur les droites 0, 5, 11 et 16. On obtient les listes suivantes :

In [4]:
Murs_horizontaux=[[0,5,11,16],[0,6,13,16],[0,4,16],[0,15,16],[0,10,16],[0,3,16],[0,10,16],[0,6,7,9,12,16],[0,7,9,16],[0,3,12,16],[0,14,16],[0,16],[0,7,16],[0,2,10,16],[0,4,13,16],[0,2,12,16]]
Murs_verticaux=[[0,4,10,16],[0,14,16],[0,6,16],[0,9,16],[0,3,15,16],[0,7,16],[0,1,12,16],[0,7,9,16],[0,7,9,16],[0,4,13,16],[0,6,16],[0,10,16],[0,8,16],[0,2,15,16],[0,4,10,16],[0,5,12,16]]

4. À l'aide de ces listes, construire la liste d'adjacence, notée `Liste_adjacence`, après un seul pas du déplacement du robot. Quelle est la taille du graphe correspondant ?

In [5]:
Liste_adjacence=[]
for indice in range(nb_cases):
    [i,j]=Sommets[indice]
    Adjacents=[]
    if not(j in Murs_verticaux[i]):
        Adjacents=Adjacents+[indice_case([i,j-1])]
    if not(j+1 in Murs_verticaux[i]):
        Adjacents=Adjacents+[indice_case([i,j+1])]
    if not(i in Murs_horizontaux[j]):
        Adjacents=Adjacents+[indice_case([i-1,j])]
    if not(i+1 in Murs_horizontaux[j]):
        Adjacents=Adjacents+[indice_case([i+1,j])]
    Liste_adjacence=Liste_adjacence+[Adjacents]

In [6]:
taille_pas=0
for indice in range(nb_cases):
    taille_pas=taille_pas+len(Liste_adjacence[indice])
taille_pas=taille_pas/2
print(taille_pas)

426.0


## Construction de la matrice d'adjacence après un mouvement entier

<br>

Le but de cette partie est de construire la matrice d'adjacence après un mouvement entier du robot (jusqu'au premier mur). Par exemple, les sommets `[1,7]` et `[1,8]` ne sont plus considérés adjacents, alors que les sommets `[1,7]` et `[1,13]` sont adjacents. Plus précisément, `[1,13]` est un successeur de `[1,7]`, alors que `[1,7]` n'est pas un successeur de `[1,13]` (mais `[1,0]` est un successeur de `[1,13]`). On remarque que la matrice d'adjacence n'est pas symétrique, donc que le graphe correspondant est orienté.

Pour modéliser les 4 directions possibles du mouvement du robot, on considère le dictionnaire suivant :

In [7]:
Directions={'nord':[-1,0],'est':[0,1],'sud':[1,0],'ouest':[0,-1]}

5. À l'aide de `Liste_adjacence`, écrire une fonction `indice_arrivee` qui prend en arguments l'indice d'une case de départ du robot et la direction de son mouvement, puis qui renvoie l'indice de sa case d'arrivée (jusqu'au premier mur).

In [8]:
def indice_arrivee(indice_depart,direction):
    indice=indice_depart
    [i,j]=Sommets[indice]
    [di,dj]=Directions[direction]
    while indice_case([i+di,j+dj]) in Liste_adjacence[indice]:
        indice=indice_case([i+di,j+dj])
        [i,j]=Sommets[indice]
    return indice

6. Construire la matrice d'adjacence, notée `Matrice_adjacence`, après un mouvement entier du robot (jusqu'au premier mur). Quelle est la taille du graphe correspondant ? Comparer ce résultat avec celui de la question 4 et justifier.

In [9]:
Matrice_adjacence=[[0 for indice2 in range(nb_cases)] for indice1 in range(nb_cases)]
for indice1 in range(nb_cases):
    for direction in list(Directions):
        indice2=indice_arrivee(indice1,direction)
        if indice2!=indice1:
            Matrice_adjacence[indice1][indice2]=1

In [10]:
taille_mouvement=0
for indice1 in range(nb_cases):
    for indice2 in range(nb_cases):
        taille_mouvement=taille_mouvement+Matrice_adjacence[indice1][indice2]
print(taille_mouvement)

852


## Détermination des cases atteignables depuis une case de départ

<br>

7. À l'aide d'un <b>parcours en profondeur</b>, écrire une fonction `liste_atteignables` qui prend en argument une case de départ du robot, puis qui renvoie la liste des cases que le robot peut atteindre en un nombre fini de mouvements.

8. Existe-t-il une case de départ du robot qui ne lui permet pas d'atteindre un objectif situé sur la case en haut à gauche du plateau ?

9. Quelles sont les cases de départ qui permettent d'atteindre le plus grand nombre de cases du plateau ?

## Calcul du nombre minimal de mouvements depuis une case de départ

<br>

10. À l'aide d'un <b>parcours en largeur</b>, écrire une fonction `liste_min_mouvements` qui prend en argument une case de départ du robot, puis qui renvoie une liste de listes dont la $n$-ième liste est la liste des cases que le robot peut atteindre en nombre minimal de mouvements égal à $n$.

Utiliser `liste_min_mouvements` pour vérifier votre résultat de la question 1.

11. Écrire une fonction `cases_eloignees`qui prend en argument une case de départ du robot, puis qui renvoie la liste des cases que le robot peut atteindre avec un nombre minimal de mouvements maximal.

12. Quelles sont les cases les plus éloignées si le robot démarre dans le coin en haut à gauche du plateau ? Combien de mouvements sont nécessaires pour les atteindre ?

## Résolution du jeu

<br>

13. À l'aide d'un <b>algorithme de Dijkstra</b>, écrire une fonction `solution` qui prend en arguments une case de départ du robot et une case objectif à atteindre, puis qui renvoie la liste des cases correspondant à un déplacement du robot résolvant le problème en utilisant le moins de mouvements possibles. Si la case objectif n'est pas atteignable depuis la case de départ, la fonction doit renvoyer `'pas de solution'`.

14. Utiliser `solution` pour déterminer des solutions partant du coin en haut à gauche du plateau jusqu'aux cases les plus éloignées de cette case de départ.

## Détermination des solutions maximales

<br>

15. À l'aide d'un <b>algorithme de Floyd-Warshall</b>, construire la matrice `Matrice_min_mouvements` des nombres minimaux de mouvements entre chaque case de départ et chaque case objectif à atteindre. Si une case objectif n'est pas atteignable depuis une case de départ, le coefficient correspondant de `Matrice_min_mouvements` doit être égal à `numpy.Infinity`.

16. Déterminer les solutions maximales du jeu, c'est-à-dire celles dont le nombre minimal de mouvements nécessaires est maximal.